## import libraries:

In [2]:
import mediapipe as mp
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import threading
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Setup Sound:

In [13]:
pygame.mixer.init()
sound=pygame.mixer.Sound(r"beep-09 2.wav")
def play_sound():
    if not pygame.mixer.get_busy():
        sound.play(-1)
        

def stop_sound():
    
    if pygame.mixer.get_busy():
        sound.stop()

## Add function for Progress bar:

In [14]:
def draw_progress_bar(frame, ear_value):
    bar_width = 250
    bar_height = 20
    bar_x = 10
    bar_y = 120
     
    min_ear = 0.23
    max_ear = 0.30
    percentage = (ear_value - min_ear) / (max_ear - min_ear)
    percentage = max(0, min(1, percentage))  
    
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (255, 255, 255), 2)
    
    filled_width = int(bar_width * percentage)
    
    #if percentage < EAR_THRESHOLD:
    if percentage<0.4:
        bar_color = (0, 0, 255) 
    else:
        bar_color = (0, 255, 0)  
    
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + filled_width, bar_y + bar_height), bar_color, -1)
    
    #EAR
    status_text = f"Eye Opening: {int(percentage * 100)}% (EAR: {ear_value:.2f})"
    cv2.putText(frame, status_text, (bar_x, bar_y - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

## Let's do that:)

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

def calculate_ear(eye_points):
    
    v1 = np.linalg.norm(eye_points[1] - eye_points[5])
    v2 = np.linalg.norm(eye_points[2] - eye_points[4])
    h = np.linalg.norm(eye_points[0] - eye_points[3])
    #  EAR
    ear = (v1 + v2) / (2.0 * h)
    return ear     # Eye Aspect Ratio


with mp_face_mesh.FaceMesh(max_num_faces=1,min_detection_confidence=0.5,
                           min_tracking_confidence=0.5) as face_mesh:
    
    EAR_THRESHOLD = 0.28
    EAR_FRAMES =40
    blink_count=0
    eye_closed=False
    start_time=None
    
    counter=0  
    video=cv2.VideoCapture(0)
    while True:
        ret, frame = video.read()
        if not ret:
            break
        H,W,_=frame.shape
        mp_face_detection=mp.solutions.face_detection
        with mp_face_detection.FaceDetection(min_detection_confidence=0.5
                                  ,model_selection=0) as face_detection:
            rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            out=face_detection.process(rgb_img)
            #print(out.detections)
            if out.detections is not None:
                for detection in out.detections:
                    location_data=detection.location_data
                    bbox=location_data.relative_bounding_box
                    x1,y1,w,h=bbox.xmin,bbox.ymin,bbox.width,bbox.height

                    x1=int(x1*W)
                    y1=int(y1*H)
                    w=int(w*W)
                    h=int(h*H)
                    cv2.rectangle(frame,(x1,y1),(x1+w,y1+h),(139,0,0),2)
                    print(f'Face_bbox(x1:{x1},y1:{y1},w:{w},h:{h})')
                    #cv2.putText(frame,'Face',(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(rgb_frame)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                #print(results.multi_face_landmarks)

                left_eye = np.array([
                    [int(face_landmarks.landmark[33].x * frame.shape[1]), int(face_landmarks.landmark[33].y * frame.shape[0])],
                    [int(face_landmarks.landmark[160].x * frame.shape[1]), int(face_landmarks.landmark[160].y * frame.shape[0])],
                    [int(face_landmarks.landmark[158].x * frame.shape[1]), int(face_landmarks.landmark[158].y * frame.shape[0])],
                    [int(face_landmarks.landmark[133].x * frame.shape[1]), int(face_landmarks.landmark[133].y * frame.shape[0])],
                    [int(face_landmarks.landmark[153].x * frame.shape[1]), int(face_landmarks.landmark[153].y * frame.shape[0])],
                    [int(face_landmarks.landmark[144].x * frame.shape[1]), int(face_landmarks.landmark[144].y * frame.shape[0])]
                ])
                
                right_eye = np.array([
                    [int(face_landmarks.landmark[362].x * frame.shape[1]), int(face_landmarks.landmark[362].y * frame.shape[0])],
                    [int(face_landmarks.landmark[387].x * frame.shape[1]), int(face_landmarks.landmark[387].y * frame.shape[0])],
                    [int(face_landmarks.landmark[386].x * frame.shape[1]), int(face_landmarks.landmark[386].y * frame.shape[0])],
                    [int(face_landmarks.landmark[263].x * frame.shape[1]), int(face_landmarks.landmark[263].y * frame.shape[0])],
                    [int(face_landmarks.landmark[373].x * frame.shape[1]), int(face_landmarks.landmark[373].y * frame.shape[0])],
                    [int(face_landmarks.landmark[380].x * frame.shape[1]), int(face_landmarks.landmark[380].y * frame.shape[0])]
                ])
        
                left_ear = calculate_ear(left_eye)
                right_ear = calculate_ear(right_eye)
                
                ear = (left_ear + right_ear) / 2.0
                draw_progress_bar(frame, ear)
                if ear>EAR_THRESHOLD:
                    eye_closed=True
                    
                else:
                    if eye_closed:
                        blink_count+=1
                    eye_closed=False
                cv2.putText(frame, f"Blinks:{blink_count}", (10, 180), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (139, 0,0), 2)


                if ear < EAR_THRESHOLD:
                    cv2.putText(frame, "Eyes Closed", (10, 30), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    if start_time is None:
                        start_time=time.time() 
                    counter +=1
                    if counter >= EAR_FRAMES:
                        elapsed_time=time.time()-start_time  
                        cv2.putText(frame, f"Eyes Closed for {elapsed_time:.1f} sec", (10, 30), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        
                        threading.Thread(target=play_sound).start()
                        
                else:
                    counter=0
                    cv2.putText(frame, "Eyes Open", (10, 30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    stop_sound()
                    start_time=None
                
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 70), 
                          cv2.FONT_HERSHEY_SIMPLEX, 1, (139,0,0), 2)
                
                
                for point in left_eye:
                    cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
                for point in right_eye:
                    cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
                
                for i in range(3):
                    cv2.line(frame, tuple(left_eye[i]), tuple(left_eye[i+3]), (0, 255, 0), 1)
                    cv2.line(frame, tuple(right_eye[i]), tuple(right_eye[i+3]), (0, 255, 0), 1)
        
        cv2.imshow('Eye State Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

video.release()
cv2.destroyAllWindows()